In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import display, HTML

import matplotlib.patches as patches
import plotly.graph_objects as go
import os
#from kaggle.api.kaggle_api_extended import KaggleApi
#@MATT This one doesnt run for me FYI
from tqdm.notebook import tqdm
from scipy.spatial.distance import cdist

In [ ]:
#Matt
def load_dataset(dataset_name):
    """
    Download a specific dataset from data directory.

    Parameters:
    - dataset_name: Name of the dataset to load
    """
    return pd.read_csv(f"C:\\Users\\mattd\\Documents\\GitHub\\big-data-bowl-2024\\data\\{dataset_name}.csv")

# Read In csvs
games = load_dataset("games")
plays = load_dataset("plays")
players = load_dataset("players")
week1 = load_dataset("tracking_week_1")
tackles = load_dataset("tackles")
joined_all = pd.merge(games,plays,how="inner",on = "gameId")
joined_all = pd.merge(joined_all,week1,how="inner",on=["gameId","playId"])
joined_all = pd.merge(joined_all,tackles,how="left",on=["gameId","playId","nflId"])
# # left join on players to keep football records
joined_all = pd.merge(joined_all,players,how="left",on = "nflId")

In [2]:
#Katie
games = pd.read_csv("games.csv")
plays = pd.read_csv("plays.csv")
players = pd.read_csv("players.csv")
week1 = pd.read_csv("tracking_week_1.csv")
tackles = pd.read_csv("tackles.csv")
joined_all = pd.merge(games,plays,how="inner",on = "gameId")
joined_all = pd.merge(joined_all,week1,how="inner",on=["gameId","playId"])
joined_all = pd.merge(joined_all,tackles,how="left",on=["gameId","playId","nflId"])
# # left join on players to keep football records
joined_all = pd.merge(joined_all,players,how="left",on = "nflId")

In [ ]:
#KATIE AND EMILY ADD HOW YOU READ IN DATA

In [4]:
pass_arrived = joined_all.loc[joined_all['event']=='pass_arrived'].copy()
pass_arrived

,gameId,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,homeFinalScore,visitorFinalScore,playId,...,tackle,assist,forcedFumble,pff_missedTackle,height,weight,birthDate,collegeName,position,displayName_y
0,2022090800,2022,1,09/08/2022,20:20:00,LA,BUF,10,31,2184,...,NaN,NaN,NaN,NaN,6-5,325.0,1988-06-06,Indiana,G,Rodger Saffold
44,2022090800,2022,1,09/08/2022,20:20:00,LA,BUF,10,31,2184,...,NaN,NaN,NaN,NaN,6-0,242.0,1990-06-27,Utah State,ILB,Bobby Wagner
88,2022090800,2022,1,09/08/2022,20:20:00,LA,BUF,10,31,2184,...,NaN,NaN,NaN,NaN,6-1,280.0,1991-05-23,Pittsburgh,DT,Aaron Donald
132,2022090800,2022,1,09/08/2022,20:20:00,LA,BUF,10,31,2184,...,NaN,NaN,NaN,NaN,6-6,305.0,1992-04-21,Missouri,C,Mitch Morse
176,2022090800,2022,1,09/08/2022,20:20:00,LA,BUF,10,31,2184,...,NaN,NaN,NaN,NaN,6-0,191.0,1993-11-29,Maryland,WR,Stefon Diggs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407316,2022091200,2022,1,09/12/2022,20:15:00,SEA,DEN,17,16,3467,...,NaN,NaN,NaN,NaN,6-2,203.0,NaN,Alabama,CB,Patrick Surtain
1407341,2022091200,2022,1,09/12/2022,20:15:00,SEA,DEN,17,16,3467,...,NaN,NaN,NaN,NaN,6-3,240.0,NaN,Ohio State,ILB,Baron Browning
1407366,2022091200,2022,1,09/12/2022,20:15:00,SEA,DEN,17,16,3467,...,NaN,NaN,NaN,NaN,6-5,310.0,NaN,Mississippi State,T,Charles Cross
1407391,2022091200,2022,1,09/12/2022,20:15:00,SEA,DEN,17,16,3467,...,NaN,NaN,NaN,NaN,6-7,319.0,NaN,Washington State,T,Abraham Lucas


In [51]:
#Tracking df/play cleaning functions : New by katie 
"""
Functions are meant to be run on a frame by frame basis. The dataframe passed into it should contain just the tracking from one play
"""


def distance_to_ball(frame):
    """
    Calculates the distance to the ball for each player per frame in a play.

    Parameters:
    - dataset_name: Name of the dataset to load
    """
    carr_x, carr_y = ball_carrier_info(frame)
    frame['distance_to_tackle'] = cdist(frame[['x', 'y']], [[carr_x, carr_y]], metric='euclidean')
    return frame


def ball_carrier_info(frame):
    """
    Returns the x and y coordinates for the ball carrier.

    Parameters:
    - frame: Name of df contained one frame from one play
    """
    ball_carrier_id = frame["ballCarrierId"].iloc[0]
    ball_carrier = frame[frame["nflId"]==ball_carrier_id]
    return ball_carrier["x"].values[0], ball_carrier["y"].values[0]


def with_additional_data(frame, players, plays):
    """
    frame: Dataframe containing one frame from one play
    players: Dataframe containing players data
    plays: Dataframe containing play data
    """
    # Selecting specific columns from players data
    players_columns = ['nflId', 'displayName', 'jerseyNumber', 'height', 'weight']
    frame = pd.merge(frame, players[players_columns], on="nflId", how="left")
    # Selecting specific columns from play data
    play_columns = ['gameId', 'playId', 'quarter', 'ballCarrierId', 'down', 'yardsToGo',
                    'yardlineNumber', 'gameClock', 'passLength', 'absoluteYardlineNumber',
                    'expectedPoints', 'defendersInTheBox']
    frame = pd.merge(frame, plays[play_columns], on=["gameId", "playId"], how="left")
    return frame



In [ ]:
#Matts old 
#Tracking df/play cleaning functions 
"""
Functions are meant to be run on a frame by frame basis. The dataframe passed into it should contain just the tracking from one play
"""

def distance_to_ball(frame):
    """
    Calculates the distance to the ball for each player per frame in a play.

    Parameters:
    - dataset_name: Name of the dataset to load
    """
    carr_x, carr_y = ball_carrier_info(frame)
    frame['distance_to_tackle'] = cdist(frame[['x', 'y']], [[carr_x, carr_y]], metric='euclidean')
    return frame



def ball_carrier_info(frame):
    """
    Returns the x and y coordinates for the ball carrier.

    Parameters:
    - frame: Name of df contained one frame from one play
    """
    ball_carrier_id = frame["ballCarrierId"].iloc[0]
    ball_carrier = frame[frame["nflId"]==ball_carrier_id]
    return ball_carrier["x"].values[0], ball_carrier["y"].values[0]


In [52]:

#new by katie 

gid = 2022091103  

joined_play_tracking = pd.merge(plays, week1, how="inner", on=["gameId", "playId"])
game = joined_play_tracking.loc[joined_play_tracking['gameId'] == gid].copy()
pass_play = game.loc[game['event'] == 'pass_arrived'].copy()
frame_test = pass_play[(pass_play['frameId'] == 4) & (pass_play['playId'] == 58)]

enriched_frame = with_additional_data(frame_test, players, plays)

enriched_frame = distance_to_ball(enriched_frame)


desired_columns = ["ballCarrierId", "nflId", "distance_to_tackle", "x", "y",
                   "displayName", "jerseyNumber", "height", "weight",
                   "quarter", "down", "yardsToGo", "yardlineNumber",
                   "gameClock", "passLength", "absoluteYardlineNumber",
                   "expectedPoints", "defendersInTheBox"]

display_frame = enriched_frame[desired_columns]
print(display_frame)

KeyError: "['jerseyNumber'] not in index"

In [53]:
#matts old 

# Testing distance_to_ball and ball_carrier_info
gid = 2022091103 #randomly sampled game 
joined_play_tracking = pd.merge(plays,week1,how="inner",on=["gameId","playId"])
game = joined_play_tracking.loc[joined_play_tracking['gameId']==gid].copy()
pass_play = game.loc[game['event']=='pass_arrived'].copy()
frame_test = pass_play[(pass_play['frameId']==4) & (pass_play['playId']==58)]

test_df = distance_to_ball(frame_test)
test_df[["ballCarrierId","nflId","Distance_to_reference","x","y"]]



/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


KeyError: "['Distance_to_reference'] not in index"